In [ ]:
"""
Samuel Diaz 
Assignment 3
CS-381
04/15/2022

**NOTE: IMPLEMETATION AT THE VERY BOTTOM**

Description: For this assignment, write program that implements a Multinomial Naive Bayes 
text classifier and test it using the movie_reviews dataset. Start with the Bernoulli Naive Bayes 
implementation from class as a model and modify it as necessary.
To submit, make sure your notebook shows the code you've written as well as output when you run it 
(including the accuracy). Save your notebook as an HTML file (using the File > Save and export notebook 
as .... > HTML menu item), and the submit the HTML file through canvas.
"""

In [ ]:
#NAIVE BAYES

In [3]:
from collections import Counter, defaultdict
from math import log
import pandas as pd
import numpy as np
from toolz import *
from nltk.corpus import movie_reviews

In [4]:
#READ DATE

In [5]:
movie_reviews.words()

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [6]:
movie_reviews.categories()

['neg', 'pos']

In [7]:
data = pd.DataFrame([{'text':list(movie_reviews.words(fileid)), 'label':category}
                     for category in movie_reviews.categories()
                     for fileid in movie_reviews.fileids(categories=category)])

In [8]:
data

,text,label
0,"[plot, :, two, teen, couples, go, to, a, churc...",neg
1,"[the, happy, bastard, ', s, quick, movie, revi...",neg
2,"[it, is, movies, like, these, that, make, a, j...",neg
3,"["", quest, for, camelot, "", is, warner, bros, ...",neg
4,"[synopsis, :, a, mentally, unstable, man, unde...",neg
...,...,...
1995,"[wow, !, what, a, movie, ., it, ', s, everythi...",pos
1996,"[richard, gere, can, be, a, commanding, actor,...",pos
1997,"[glory, --, starring, matthew, broderick, ,, d...",pos
1998,"[steven, spielberg, ', s, second, epic, film, ...",pos


In [9]:
#BASELINE

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [11]:
train, test = train_test_split(data, train_size=0.75, random_state=581)

In [12]:
train

,text,label
1206,"[you, ', ve, got, to, love, disney, ., no, mat...",pos
1898,"[a, cinematic, version, of, one, of, john, irv...",pos
1945,"[there, is, a, striking, scene, early, in, "", ...",pos
364,"[dear, god, ,, this, is, a, fantastically, bad...",neg
1075,"[well, ,, i, know, that, stallone, is, 50, yea...",pos
...,...,...
987,"[please, don, ', t, mind, this, windbag, letti...",neg
11,"[janeane, garofalo, in, a, romantic, comedy, -...",neg
1345,"[alien, 3, is, the, only, alien, film, i, have...",pos
413,"[there, ', s, a, scene, somewhere, in, this, f...",neg


In [13]:
dummy = DummyClassifier()
dummy.fit(train['text'], train['label'])
accuracy_score(test['label'], dummy.predict(test['text']))

0.498

In [14]:
#BERNOULI BN

In [31]:
class BNB:
    
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        
    def fit(self, texts, labels):
        
        self.vocab = set()
        
        # Pc[c] = class prior prob. P(c)
        self.Pc = Counter()
        
        # P[wc] = word likelihood P(w|c)
        self.Pwc = defaultdict(Counter)
        
        for text, label in zip(texts, labels):
            self.Pc[label] += 1
            for w in set(text):
                self.vocab.add(w)
                self.Pwc[label][w] += 1                
        N = sum(self.Pc.values())
        
        for c in self.Pc:
            for w in self.vocab:
                self.Pwc[c][w] = (self.Pwc[c][w] + self.alpha) / (self.Pc[c] + self.alpha * 2)
            self.Pc[c] = self.Pc[c] / N
            
    def predict(self, texts):
        labels = [ ]
        for text in texts:
            text = set(text)
            c_max = None
            p_max = -np.infty
            for c in self.Pc:
                p = log(self.Pc[c])
                for w in self.vocab:
                    if w in text:
                        p += log(self.Pwc[c][w])
                    else:
                        p += log(1-self.Pwc[c][w])
                if p > p_max:
                    p_max = p
                    c_max = c
            labels.append(c_max)
        return labels

In [32]:
nb = BNB(alpha=1.0)
nb.fit(train['text'], train['label'])

In [ ]:
accuracy_score(test['label'], nb.predict(test['text']))

0.762

In [18]:
#scikit-learn classifiers

In [19]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

In [20]:
skbnb = make_pipeline(CountVectorizer(binary=True, analyzer=identity), BernoulliNB(alpha=1.0))

In [21]:
skbnb.fit(train['text'], train['label'])

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(analyzer=<function identity at 0x2b0bc2573a60>,
                                 binary=True)),
                ('bernoullinb', BernoulliNB())])

In [22]:
accuracy_score(test['label'], skbnb.predict(test['text']))

0.762

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
skmnb = make_pipeline(CountVectorizer(analyzer=identity), MultinomialNB(alpha=1.0))
skmnb.fit(train['text'], train['label'])
accuracy_score(test['label'], skmnb.predict(test['text']))

0.796

In [25]:
from sklearn.linear_model import SGDClassifier

In [26]:
sksgd = make_pipeline(CountVectorizer(analyzer=identity), SGDClassifier(loss='log'))
sksgd.fit(train['text'], train['label'])
accuracy_score(test['label'], sksgd.predict(test['text']))

0.756

In [27]:
#Multinomial Naive Bayes

In [328]:
class MultinomialNB:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        
    def fit(self, texts, labels):
        self.vocab = set()
        
        # Pc[c] = class prior prob. P(c)
        self.Pc = Counter()
        
        # P[wc] = word likelihood P(w|c)
        self.Pwc = defaultdict(Counter)
        
        for text, label in zip(texts, labels):
            vocabulary_size=0
            self.Pc[label] += 1
            for w in set(text):
                self.vocab.add(w)
                self.Pwc[label][w] += 1
            for w in list(text):
                vocabulary_size+=1
        N = sum(self.Pc.values())
        V = vocabulary_size

        for c in self.Pc:
            for w in self.vocab:
                    self.Pwc[c][w] = (self.Pwc[c][w] + self.alpha) / (self.Pc[c] + self.alpha * abs(V))
            self.Pc[c] = self.Pc[c] / N
            
    def predict(self, texts):
        labels = [ ]
        for text in texts:
            text = set(text)
            c_max = None
            p_max = -np.infty
            for c in self.Pc:
                p = log(self.Pc[c])
                for w in self.vocab:
                    if w in text:
                        p += log(self.Pwc[c][w])
                    else:
                        try:
                            p += log(1-self.Pwc[c][w])
                        except ValueError:
                            continue
                if p > p_max:
                    p_max = p
                    c_max = c
            labels.append(c_max)
        return labels

In [329]:
nb = MultinomialNB(alpha=1.0)
nb.fit(train['text'], train['label'])

In [330]:
accuracy_score(test['label'], nb.predict(test['text']))

0.778